In [ ]:

from cartopy import config
import cartopy
import cartopy.crs as ccrs
import climtas
import cmocean
from collections import namedtuple
import dask.array
from datetime import datetime
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import scipy.stats
from scipy import stats
from scipy.stats import ttest_ind 
import seaborn as sns
import sparse
from string import ascii_lowercase
import xarray as xr


import warnings
warnings.filterwarnings("ignore")
import netCDF4 as nc
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr    
import glob
import os
import cartopy.crs as ccrs
import matplotlib.colors as colors
import pandas as pd
import datetime
from matplotlib import colors
import pymannkendall as mk
from matplotlib.ticker import MaxNLocator

In [ ]:
cmap2 = plt.cm.get_cmap("BuGn", 15)

In [ ]:
HWX_list =["HWA_EHF","HWM_EHF","HWN_EHF","HWF_EHF","HWD_EHF","HWT_EHF"]
vmaxlist=[40,20,3.5,17.5,9,80]
vminlist=[0,0,1.5,5,3,40]
land_sea = xr.open_dataset("/g/data/w97/sl7808/landmask_accessESM1-5_AUS.nc").sftlf

        
#plist=[0,0.25,0.5,1,1.5,2]
#nlist=[0,-0.25,-0.5,-1,-1.5,-2]



list2=[1,2,3,4,5,6]



#DMI
d_filenames = sorted(glob.glob("/g/data/w97/sl7808/ALLOUTPUT/DMI_ACCESS/*.nc")) 

file_name = []
f=[]
for d in d_filenames:   
    
    a = xr.open_dataset(d)
    DMI_p=a.sel(time=a.time.dt.month.isin([8,9,10])) #select DMI peak months
    x=DMI_p.tos.groupby('time.year').mean(dim='time')#take the mean of each year
    z=x.rename({'year':'time'})
    del z['time']
    f.insert(-1,z)

pIOD_1=xr.concat(f, 'ensemble')
pIOD=pIOD_1

In [ ]:
nino_Access_filenames = []
nino_Access_filenames = sorted(glob.glob("/g/data/w97/sl7808/ALLOUTPUT/Nino34_ACCESS/all20_nor/*.nc")) 
file_name = []
f=[]


for n in nino_Access_filenames:
        
    a = xr.open_dataset(n)
    a1=a.sel(time=a.time.dt.month.isin([6,7,8,9,10,11])).tos.groupby('time.year').mean(dim='time')#select ENSO peak months
    a2=a1.rename({'year':'time'})
    del a2['time']
    f.insert(-1,a2)

nino=xr.concat(f, 'ensemble')
nino

In [ ]:
def choose_H(): 
    if num==0:
        HH=H.where(pIOD_sel>p).where(nino_sel>p).mean(dim=['time'])
        g.insert(-1,HH)
    elif num==1:
        HH=H.where(pIOD_sel>p).where(nino_sel>n).where(nino_sel<p).mean(dim=['time'])
        g.insert(-1,HH)
    elif num==2:
        HH=H.where(pIOD_sel>p).where(nino_sel<n).mean(dim=['time'])
        g.insert(-1,HH)
    elif num==3:
        HH=H.where(pIOD_sel>n).where(pIOD_sel<p).where(nino_sel>p).mean(dim=['time'])
        g.insert(-1,HH)
    elif num==4:
        HH=H.where(pIOD_sel>n).where(pIOD_sel<p).where(nino_sel>n).where(nino_sel<p).mean(dim=['time'])
        g.insert(-1,HH)
    elif num==5:
        HH=H.where(pIOD_sel>n).where(pIOD_sel<p).where(nino_sel<n).mean(dim=['time'])
        g.insert(-1,HH)
    elif num==6:
        HH=H.where(pIOD_sel<n).where(nino_sel>p).mean(dim=['time'])
        g.insert(-1,HH)
    elif num==7:
        HH=H.where(pIOD_sel<n).where(nino_sel>n).where(nino_sel<p).mean(dim=['time'])
        g.insert(-1,HH)
    elif num==8:
        HH=H.where(pIOD_sel<n).where(nino_sel<n).mean(dim=['time'])
        g.insert(-1,HH)
        

In [ ]:

p=1
n=-1



#klist=[H1,H2,H3,H4,H5,H6,H7,H8,H9]

kklist_1=[0,0,0,1,1,1,2,2,2]
kklist_2=[0,1,2,0,1,2,0,1,2]


for (h,xxx,nnn) in zip (HWX_list,vmaxlist,vminlist):
    #HWX
    fig, axes = plt.subplots(3,3, figsize=(15,15), subplot_kw={'projection': ccrs.PlateCarree(central_longitude=0)})

    

    for i in range(3):
        for j in range(3):
            axes[j,i].axis('off') 
            axes[j,i].coastlines()
            

    
    
    d_filenames = []
    d_filenames = sorted(glob.glob("/g/data/w97/sl7808/ALLOUTPUT/EHF_year_ACCESS/ehf_year_exp*.nc"))   
    f=[]
    for d in d_filenames:   
        a = xr.open_dataset(d, decode_times=False)
        a1 = a[h]
        a2=a1.where(a1>0)
        f.insert(-1,a2)

    HWA=xr.concat(f, 'ensemble')

    
    
    g=[]
    ggg=[]
    #klist=[H1,H2,H3,H4,H5,H6,H7,H8,H9]
    num=0
    for k1,k2 in zip (kklist_1,kklist_2):
        g=[]
        ggg=[]
        
        for i in range(20): 
            nino_sel=nino.isel(ensemble=i)
            pIOD_sel=pIOD.isel(ensemble=i)
            H=HWA.isel(ensemble=i).where(land_sea>60)
            choose_H()
        ggg=xr.concat(g, 'ensemble')  
        #print(ggg)
        ggg.mean(dim='ensemble').plot(ax=axes[k1,k2],vmax=xxx,vmin=nnn,cmap=cmap2, add_colorbar=False, transform=ccrs.PlateCarree())
        num=num+1

    

    # plot
    
    

    
    
    num=1
    
    
        
    #Nino+, DMI+
    c=nino.where(pIOD_sel>p).where(nino_sel>p).count()
    axes[0,0].set_title( 'Nino+, DMI+' + '  ('+str(c.values)+')', fontsize=10)

    
    #Nino neu, DMI+

    c=nino.where(pIOD_sel>p).where(nino_sel>n).where(nino_sel<p).count()
    axes[0,1].set_title( 'Nino ||, DMI +' + '  ('+str(c.values)+')', fontsize=10) 
    
    
    #Nino-, DMI+
    c=nino.where(pIOD_sel>p).where(nino_sel<n).count()
    axes[0,2].set_title( 'Nino+, DMI-' + '  ('+str(c.values)+')', fontsize=10)      
    
    
    #DMI neutral, Nino +
    c=nino.where(pIOD_sel>n).where(pIOD_sel<p).where(nino_sel>p).count()
    axes[1,0].set_title( 'Nino +, DMI ||' + '  ('+str(c.values)+')', fontsize=10)     
    
    #Both neutral
    c=nino.where(pIOD_sel>n).where(pIOD_sel<p).where(nino_sel>n).where(nino_sel<p).count()
    axes[1,1].set_title( 'Nino ||, DMI ||' + '  ('+str(c.values)+')', fontsize=10)           
    
    
    
    #DMI Neu, Nino -
    c=nino.where(pIOD_sel>n).where(pIOD_sel<p).where(nino_sel<n).count()
    axes[1,2].set_title( 'Nino -, DMI ||' + '  ('+str(c.values)+')', fontsize=10)           
  
                                     
 
    
    #Nino+, DMI-
    c=nino.where(pIOD_sel<n).where(nino_sel>p).count()
    axes[2,0].set_title( 'Nino+, DMI-' + '  ('+str(c.values)+')', fontsize=10)

    
    #DMI -, Nino Neu
    c=nino.where(pIOD_sel<n).where(nino_sel>n).where(nino_sel<p).count()
    axes[2,1].set_title( 'Nino ||, DMI -' + '  ('+str(c.values)+')', fontsize=10)           
    
  
 
    

    #Nino-, DMI-
    c=nino.where(pIOD_sel<n).where(nino_sel<n).count()
    axes[2,2].set_title( 'Nino-, DMI-' + '  ('+str(c.values)+')', fontsize=10)                
                                  
 
                                      
                                  
  
    
    #plott=HWA.where(land_sea>60).mean(dim=['time','ensemble']).plot(ax=axes[0,0],vmax=xxx,vmin=nnn,cmap=cmap2, add_colorbar=False)
    #c=nino_Access.count()
    #axes[0,0].set_title("Nino3.4 full range"+'  ('+str(c.values)+')', fontsize=10)
    #axes[0,0].coastlines()
    #fig.colorbar(plott, ax=axes[2,0], orientation='horizontal',extend='both',label=h)

    #fig.delaxes(axes[1,1])
    fig.subplots_adjust(wspace=0, hspace=0)
    fig.suptitle(h + '  all years', fontsize=16)
